## Covid World Map Geovisualization

This project aims to make a World Map Geovisualization of Covid Data using the Folium Library. The data is sourced from "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University" (updated daily). This project was realized with data up-to-date to the 12th of September 2020.

### Import Libraries

In [9]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [10]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

### Convert Data to JSON

In [11]:
covid1 = json.loads(data)

### Normalize Data 

In [12]:
pd.json_normalize(covid1['Countries'], sep = ",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,34,38606,0,1420,25,31154,2020-09-12T07:29:45Z
1,Albania,AL,albania,161,11021,3,327,97,6443,2020-09-12T07:29:45Z
2,Algeria,DZ,algeria,264,47752,8,1599,161,33723,2020-09-12T07:29:45Z
3,Andorra,AD,andorra,43,1344,0,53,5,943,2020-09-12T07:29:45Z
4,Angola,AO,angola,62,3279,1,131,11,1288,2020-09-12T07:29:45Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,1,1060,0,35,9,902,2020-09-12T07:29:45Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-09-12T07:29:45Z
185,Yemen,YE,yemen,4,2007,2,582,0,1211,2020-09-12T07:29:45Z
186,Zambia,ZM,zambia,109,13323,6,306,0,11899,2020-09-12T07:29:45Z


### Convert to Pandas DataFrame

In [14]:
df = pd.DataFrame(covid1['Countries'])
df.head(3)

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,34,38606,0,1420,25,31154,2020-09-12T07:29:45Z,{}
1,Albania,AL,albania,161,11021,3,327,97,6443,2020-09-12T07:29:45Z,{}
2,Algeria,DZ,algeria,264,47752,8,1599,161,33723,2020-09-12T07:29:45Z,{}


### Drop Unnecessary Columns

In [15]:
    covid2 = df.drop(columns = ['CountryCode', 'Slug', 'Date', 'Premium'], axis = 1)
    covid2.head(3) 

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,34,38606,0,1420,25,31154
1,Albania,161,11021,3,327,97,6443
2,Algeria,264,47752,8,1599,161,33723


### Generate Base Map

In [16]:
m = folium.Map(tiles = "Stamen Terrain", min_zoom = 1.5)
m

### Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [17]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

### Generate Choropleth Map Layer

In [18]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 2,
    name = 'Covid19',
    data = covid2,
    columns = ['Country', 'TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nan_fill_color = 'black',
    legend_name = 'Total Confirmed Covid Cases',
).add_to(m)
m

### Generate Circular Markers

In [19]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv

In [21]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates.head(3)

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria


In [22]:
covid_final = pd.merge(covid2, coordinates, on = 'Country')

In [23]:
def plotDot(point):
    folium.CircleMarker(location = [point.latitude, point.longitude],
                        radius = 5, 
                        weight = 2,
                        popup = [point.Country, point.TotalConfirmed, point. TotalRecovered],
                        fill_color = '#000000').add_to(m)

In [24]:
covid_final.apply(plotDot, axis = 1)
m.fit_bounds(m.get_bounds())
m

### Generate Base Map

In [25]:
m1 = folium.Map(tiles = 'StamenToner', min_zoom = 2)
m1

In [26]:
#Convert variables
deaths = covid_final['TotalDeaths'].astype(float)
lon = covid_final['longitude'].astype(float)
lat = covid_final['latitude'].astype(float)

### Generate Heat Map Layer

In [29]:
m1.add_child(HeatMap(zip(lat, lon, deaths), radius = 0))

### Generate Circular Marker

In [30]:
def plotDot(point) : 
    folium.CircleMarker(location = [point.latitude, point.longitude],
                       radius = 5,
                       weight = 2,
                       popup = [point.Country, point.TotalDeaths],
                       fill_color = '#000000').add_to(m1)
covid_final.apply(plotDot, axis = 1)
m1.fit_bounds(m1.get_bounds())
m1